# Подготовка данных

In [34]:
import re

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

## Загрузка данных

In [119]:
dataframe = pd.read_csv("../data_anonymized_with_user_ids.csv")

In [ ]:
dataframe.head()

In [ ]:
dataframe.info()

In [120]:
print(dataframe["Format"].unique())
print(dataframe["town"].unique())

['Оффлайн' 'Трейд Ин']
['АСТАНА']


In [121]:
dataframe = dataframe.drop(columns=["FirstName"])
dataframe = dataframe.drop(columns=["town"])
dataframe = dataframe.drop(columns=["FullName"])
dataframe.head()

,user_id,countPoints,Item,Date,Price,Format
0,USER_923730,335,94140036 Серьги (Ag 925),2024-09-25,28 000,Оффлайн
1,USER_923730,335,94020535 Серьги (Ag 925),2024-09-25,11 199,Оффлайн
2,USER_215040,655,94026740 Серьги (Ag 925),2024-11-14,21 860,Оффлайн
3,USER_313086,378,10-0179_ам_175 Кольцо (Ag 925),2024-06-21,4 511,Оффлайн
4,USER_313086,378,11-0075_АМ Серьги (Ag 925),2024-06-21,7 088,Оффлайн


## Кодирование колонки Format

In [122]:
one_hot_enc = OneHotEncoder(sparse_output=False)
encoded_array = one_hot_enc.fit_transform(dataframe[["Format"]])

encoded_df = pd.DataFrame(
    encoded_array, columns=one_hot_enc.get_feature_names_out(["Format"])
)
dataframe = pd.concat([dataframe.drop(columns=["Format"]), encoded_df], axis=1)

In [123]:
dataframe.rename(
    columns={"Format_Оффлайн": "Fromat_Offline", "Format_Трейд Ин": "Format_Trade_In"},
    inplace=True,
)

## Перевод колонок в тип int

In [124]:
dataframe["Price"] = dataframe["Price"].apply(
    lambda p: int("".join(filter(str.isdigit, p))) if not p.isnumeric() else int(p)
)
dataframe["countPoints"] = dataframe["countPoints"].apply(
    lambda p: int("".join(filter(str.isdigit, p))) if not p.isnumeric() else int(p)
)

In [125]:
dataframe["Item"].unique()

array(['94140036 Серьги (Ag 925)', '94020535 Серьги (Ag 925)',
       '94026740 Серьги (Ag 925)', ..., '94026865 Серьги (Ag 925)',
       '94071056_40 Колье (Ag 925)', 'LUC013 Подвеска (Au 585)'],
      shape=(17768,), dtype=object)

## Удаление лишних символов у товаров

In [128]:
products_type = [
    "Серьги", "Серьга", "Кольцо", "Обручальное кольцо", "Цепь", "Подвеска",
    "Браслет", "Коробка универсальная черная SOKOLOV", "Колье",
    "Брошь", "сертификат", "Пирсинг", "Булавка", "Сюрприз-бокс",
    "Коробка универсальная белая SOKOLOV", "Часы наручные",
    "Пакет малый бумажный черный SOKOLOV",
    "Пакет малый бумажный белый SOKOLOV", "Ложка", "Шарм",
    "Часы ювелирные", "Коробка красная квадратная",
    "Подарочная подвеска", "Запонки", "Коробка черная квадратная",
    "Сувенир", "Столовый прибор", "Коробка красная длинная", "Шнур",
    "Удлинитель", "Зажимы для галстука", "Кружка", "Брелок",
    "Вилка"
]

pattern_products = "|".join([re.escape(item) for item in products_type])

def clean_item(text):
    if pd.isna(text):
        return text

    text = str(text)

    # 1. Удаляем всё спереди до типа товара
    match = re.search(rf"({pattern_products})", text, flags=re.IGNORECASE)
    if match:
        start_idx = match.start()
        text = text[start_idx:]  # оставляем всё от типа товара до конца строки

    # 2. Убираем лишние пробелы и _
    text = re.sub(r"[_]+", " ", text)
    text = re.sub(r"\s{2,}", " ", text)

    # 3. Убираем суммы в конце строки (цифры с пробелами и \xa0)
    text = re.sub(r"(?:\s*\d[\d\s\xa0]*)$", "", text)

    # 4. Очищаем лишние точки/дефисы/пробелы с начала и конца
    text = text.strip(".- ")

    return text

In [129]:
dataframe["Item"] = dataframe["Item"].apply(clean_item)
dataframe["Item"].unique()

array(['Серьги (Ag 925)', 'Кольцо (Ag 925)',
       'Обручальное кольцо (Au 585)', 'Цепь (Ag 925)', 'Кольцо (Au 585)',
       'Серьги (Au 585)', 'Подвеска (Ag 925)', 'Браслет (Ag 925)',
       'Подвеска (Au 585)', 'Цепь (Au 585)', 'Браслет (Au 585)',
       'Коробка универсальная черная SOKOLOV', 'Колье (Ag 925)',
       'Колье (Au 585)', 'Обручальное кольцо (Ag 925)', 'Брошь (Ag 925)',
       'сертификат (6 месяцев)', 'Пирсинг (Au 585)', 'Булавка (Ag 925)',
       'Сюрприз-бокс (ювелирное изделие)', 'Серьга (Ag 925)',
       'Коробка универсальная белая SOKOLOV', 'Часы наручные (Сталь 0)',
       'Пакет малый бумажный черный SOKOLOV',
       'Пакет малый бумажный белый SOKOLOV', 'Ложка (Ag 925)',
       'Шарм (Ag 925)', 'Часы ювелирные (Au 585)', 'Брошь (Au 585)',
       'Коробка красная квадратная', 'Пирсинг (Ag 925)',
       'Подарочная подвеска (Ag 800)', 'Подарочная подвеска (Ag 925)',
       'Запонки (Ag 925)', 'Серьга (Au 585)', 'Коробка черная квадратная',
       'сертификат', 

## Подсчет материалов в item

In [130]:
def count_occurrences(data, patterns):
    counts = {p: 0 for p in patterns}

    for item in data:
        if isinstance(item, str):
            lower_item = item.lower()
            for p in patterns:
                if p.lower() in lower_item:
                    counts[p] += 1

    return counts

In [131]:
patterns = ["Сталь 0", "Ag", "Au", "ювелирное изделие", "месяц"]
materials = dataframe["Item"].tolist()

result = count_occurrences(materials, patterns)
print(result)

{'Сталь 0': 303, 'Ag': 22147, 'Au': 12791, 'ювелирное изделие': 2019, 'месяц': 229}


In [132]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39480 entries, 0 to 39479
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          39480 non-null  object 
 1   countPoints      39480 non-null  int64  
 2   Item             39480 non-null  object 
 3   Date             39480 non-null  object 
 4   Price            39480 non-null  int64  
 5   Fromat_Offline   39480 non-null  float64
 6   Format_Trade_In  39480 non-null  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 2.1+ MB


In [133]:
a = [i for i in dataframe["Item"] if "длинная" in i]
print(a)

['Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная', 'Коробка красная длинная']


## Отделяем материалы от item

In [67]:
dataframe = pd.read_csv("../prepared_data.csv")

In [ ]:
dataframe.head()

In [134]:
def extract_material_full(item):
    if pd.isna(item):
        return None

    match = re.search(r"\(([^)]+)\)", item)
    if match:
        return match.group(1).strip()
    return None


dataframe["Material"] = dataframe["Item"].apply(extract_material_full)

In [135]:
dataframe.head()

,user_id,countPoints,Item,Date,Price,Fromat_Offline,Format_Trade_In,Material
0,USER_923730,335,Серьги (Ag 925),2024-09-25,28000,1.0,0.0,Ag 925
1,USER_923730,335,Серьги (Ag 925),2024-09-25,11199,1.0,0.0,Ag 925
2,USER_215040,655,Серьги (Ag 925),2024-11-14,21860,1.0,0.0,Ag 925
3,USER_313086,378,Кольцо (Ag 925),2024-06-21,4511,1.0,0.0,Ag 925
4,USER_313086,378,Серьги (Ag 925),2024-06-21,7088,1.0,0.0,Ag 925


In [136]:
dataframe["Material"].unique()

array(['Ag 925', 'Au 585', None, '6 месяцев', 'ювелирное изделие',
       'Сталь 0', 'Ag 800', 'Au 375'], dtype=object)

In [138]:
dataframe["Item"].unique()

array(['Серьги (Ag 925)', 'Кольцо (Ag 925)',
       'Обручальное кольцо (Au 585)', 'Цепь (Ag 925)', 'Кольцо (Au 585)',
       'Серьги (Au 585)', 'Подвеска (Ag 925)', 'Браслет (Ag 925)',
       'Подвеска (Au 585)', 'Цепь (Au 585)', 'Браслет (Au 585)',
       'Коробка универсальная черная SOKOLOV', 'Колье (Ag 925)',
       'Колье (Au 585)', 'Обручальное кольцо (Ag 925)', 'Брошь (Ag 925)',
       'сертификат (6 месяцев)', 'Пирсинг (Au 585)', 'Булавка (Ag 925)',
       'Сюрприз-бокс (ювелирное изделие)', 'Серьга (Ag 925)',
       'Коробка универсальная белая SOKOLOV', 'Часы наручные (Сталь 0)',
       'Пакет малый бумажный черный SOKOLOV',
       'Пакет малый бумажный белый SOKOLOV', 'Ложка (Ag 925)',
       'Шарм (Ag 925)', 'Часы ювелирные (Au 585)', 'Брошь (Au 585)',
       'Коробка красная квадратная', 'Пирсинг (Ag 925)',
       'Подарочная подвеска (Ag 800)', 'Подарочная подвеска (Ag 925)',
       'Запонки (Ag 925)', 'Серьга (Au 585)', 'Коробка черная квадратная',
       'сертификат', 

In [139]:
patterns = ["Сталь 0", "Ag", "Au", "ювелирное изделие", "6 месяц"]
materials = dataframe["Material"].tolist()

result = count_occurrences(materials, patterns)
print(result)

{'Сталь 0': 303, 'Ag': 22147, 'Au': 12791, 'ювелирное изделие': 2019, '6 месяц': 229}


In [140]:
dataframe["Item"] = dataframe["Item"].str.replace(r"\s*\([^)]*\)", "", regex=True).str.strip()

In [141]:
dataframe["Material"] = dataframe["Material"].fillna("Other")

In [142]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39480 entries, 0 to 39479
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          39480 non-null  object 
 1   countPoints      39480 non-null  int64  
 2   Item             39480 non-null  object 
 3   Date             39480 non-null  object 
 4   Price            39480 non-null  int64  
 5   Fromat_Offline   39480 non-null  float64
 6   Format_Trade_In  39480 non-null  float64
 7   Material         39480 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 2.4+ MB


In [143]:
patterns = ["Сталь 0", "Ag", "Au", "ювелирное изделие", "6 месяц", "Other"]
materials = dataframe["Material"].tolist()

result = count_occurrences(materials, patterns)
print(result)

{'Сталь 0': 303, 'Ag': 22147, 'Au': 12791, 'ювелирное изделие': 2019, '6 месяц': 229, 'Other': 1991}


In [144]:
dataframe.head()

,user_id,countPoints,Item,Date,Price,Fromat_Offline,Format_Trade_In,Material
0,USER_923730,335,Серьги,2024-09-25,28000,1.0,0.0,Ag 925
1,USER_923730,335,Серьги,2024-09-25,11199,1.0,0.0,Ag 925
2,USER_215040,655,Серьги,2024-11-14,21860,1.0,0.0,Ag 925
3,USER_313086,378,Кольцо,2024-06-21,4511,1.0,0.0,Ag 925
4,USER_313086,378,Серьги,2024-06-21,7088,1.0,0.0,Ag 925


In [145]:
encoded_array = one_hot_enc.fit_transform(dataframe[["Material"]])

encoded_df = pd.DataFrame(
    encoded_array, columns=one_hot_enc.get_feature_names_out(["Material"])
)
dataframe = pd.concat([dataframe.drop(columns=["Material"]), encoded_df], axis=1)
dataframe.head()

,user_id,countPoints,Item,Date,Price,Fromat_Offline,Format_Trade_In,Material_6 месяцев,Material_Ag 800,Material_Ag 925,Material_Au 375,Material_Au 585,Material_Other,Material_Сталь 0,Material_ювелирное изделие
0,USER_923730,335,Серьги,2024-09-25,28000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,USER_923730,335,Серьги,2024-09-25,11199,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,USER_215040,655,Серьги,2024-11-14,21860,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,USER_313086,378,Кольцо,2024-06-21,4511,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,USER_313086,378,Серьги,2024-06-21,7088,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [146]:
dataframe.rename(
    columns={
        "Material_6 месяцев": "sertificate_6_months",
        "Material_Ag 800": "Ag_800",
        "Material_Ag 925": "Ag_925",
        "Material_Au 375": "Au_375",
        "Material_Au 585": "Au_585",
        "Material_Other": "Other_materials",
        "Material_Сталь 0": "Steal_0",
        "Material_ювелирное изделие": "Jewelry",
    },
    inplace=True,
)

In [147]:
dataframe.head()

,user_id,countPoints,Item,Date,Price,Fromat_Offline,Format_Trade_In,sertificate_6_months,Ag_800,Ag_925,Au_375,Au_585,Other_materials,Steal_0,Jewelry
0,USER_923730,335,Серьги,2024-09-25,28000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,USER_923730,335,Серьги,2024-09-25,11199,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,USER_215040,655,Серьги,2024-11-14,21860,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,USER_313086,378,Кольцо,2024-06-21,4511,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,USER_313086,378,Серьги,2024-06-21,7088,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [148]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39480 entries, 0 to 39479
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               39480 non-null  object 
 1   countPoints           39480 non-null  int64  
 2   Item                  39480 non-null  object 
 3   Date                  39480 non-null  object 
 4   Price                 39480 non-null  int64  
 5   Fromat_Offline        39480 non-null  float64
 6   Format_Trade_In       39480 non-null  float64
 7   sertificate_6_months  39480 non-null  float64
 8   Ag_800                39480 non-null  float64
 9   Ag_925                39480 non-null  float64
 10  Au_375                39480 non-null  float64
 11  Au_585                39480 non-null  float64
 12  Other_materials       39480 non-null  float64
 13  Steal_0               39480 non-null  float64
 14  Jewelry               39480 non-null  float64
dtypes: float64(10), int

In [118]:
dataframe.to_csv("../prepared_data.csv", index=False)

## Функции для визуализации

In [ ]:
def time_series_plot(data, user=None):
    ts_rolling = data.rolling(window=7).sum()
    plt.figure(figsize=(10, 5))
    plt.plot(ts_rolling.index, ts_rolling.values, color="blue", marker="o")
    plt.xticks(ts_rolling.index[::7], rotation=45)

    if user == None:
        plt.title(f"Покупки пользователя {user}")
    plt.title(f"Общие покупки пользователей")
    plt.xlabel("Дата")
    plt.ylabel("Количество купленного товара")
    plt.legend()

    plt.grid(True)
    plt.show()


def boxplot_graphic(data):
    fig, axes = plt.subplots(2, 6, figsize=(22, 11))
    fig.suptitle("Box-plot")

    row = 0
    col = 0

    for ax, feature in enumerate(data):
        data[feature].plot.box(ax=axes[row, col])
        col += 1
        if col > 5:
            row += 1
            col = 0

    plt.show()

## Временные ряды

In [ ]:
user_counts = dataframe["user_id"].value_counts()
most_active_user = user_counts.idxmax()
num_activities = user_counts.max()
print(
    f"Самый активный пользователь: {most_active_user}\nКоличество активностей: {num_activities}"
)

In [ ]:
min_ts = min(dataframe["Date"])
max_ts = max(dataframe["Date"])
print(min_ts)
print(max_ts)

In [ ]:
max_count_of_user = len(dataframe["user_id"].unique())
max_count_of_user

In [ ]:
def user_purchase(user_id):
    user_data = dataframe[dataframe["user_id"] == user_id]
    ts = user_data.groupby("Date").size()
    return ts


def common_purchase(data):
    data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
    ts = data.groupby("Date").size()
    return ts

In [ ]:
res = common_purchase(dataframe)
print(res.head(10))

time_series_plot(data=res)

In [ ]:
dataframe["Format"].hist(figsize=(8, 8))